In [1]:
import socket
import struct
import zlib
import tempfile
import tensorflow as tf
import numpy as np
from tqdm import tqdm
from sklearn.metrics import classification_report
# from tensorflow.keras.saving import register_keras_serializable
from tensorflow.keras import layers, initializers, backend as K
from sklearn.metrics import confusion_matrix, accuracy_score
from cryptography.fernet import Fernet
import base64
import hashlib

In [2]:
from tensorflow.keras import layers, initializers, backend as K

# **Custom Capsule Layers**
class Length(layers.Layer):
    def call(self, inputs, **kwargs):
        return K.sqrt(K.sum(K.square(inputs), -1) + K.epsilon())

    def compute_output_shape(self, input_shape):
        return input_shape[:-1]

class CapsuleLayer(layers.Layer):
    def __init__(self, num_capsule, dim_capsule, routings=3, **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings

    def build(self, input_shape):
        self.input_num_capsule = input_shape[1]
        self.input_dim_capsule = input_shape[2]
        self.W = self.add_weight(
            shape=[1, self.input_num_capsule, self.num_capsule, self.dim_capsule, self.input_dim_capsule],
            initializer=initializers.glorot_uniform(),
            name='W'
        )
        self.built = True

    def call(self, inputs):
        inputs_expand = K.expand_dims(K.expand_dims(inputs, 2), 2)
        W_tiled = K.tile(self.W, [K.shape(inputs)[0], 1, 1, 1, 1])
        inputs_hat = tf.squeeze(tf.matmul(W_tiled, inputs_expand, transpose_b=True), axis=-1)
        b = tf.zeros(shape=[K.shape(inputs)[0], self.input_num_capsule, self.num_capsule])

        for i in range(self.routings):
            c = tf.nn.softmax(b, axis=2)
            outputs = tf.reduce_sum(inputs_hat * K.expand_dims(c, -1), axis=1)
            if i < self.routings - 1:
                b += tf.reduce_sum(inputs_hat * K.expand_dims(c, -1), axis=-1)
        return outputs

# **Build MobileNet + Capsule Model**
class MobileNetCapsNet:
    def __init__(self, input_shape=(224, 224, 3)):
        self.input_shape = input_shape
        self.model = self._build_model()

    def _build_model(self):
        base_model = tf.keras.applications.MobileNetV2(
            input_shape=self.input_shape,
            include_top=False,
            weights='imagenet'
        )
        base_model.trainable = False  # Freeze MobileNet

        x = base_model.output
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Reshape((-1, 64))(x)
        x = CapsuleLayer(num_capsule=2, dim_capsule=8, routings=1)(x)
        outputs = Length()(x)

        return tf.keras.Model(inputs=base_model.input, outputs=outputs)

# **Margin Loss Function**
def margin_loss(y_true, y_pred):
    y_true = tf.one_hot(tf.cast(y_true, tf.int32), depth=2)
    L = y_true * tf.square(tf.maximum(0., 0.9 - y_pred)) + \
        0.5 * (1 - y_true) * tf.square(tf.maximum(0., y_pred - 0.1))
    return tf.reduce_mean(tf.reduce_sum(L, axis=1))

In [3]:
# custom_objects = {
#     'CapsuleLayer': CapsuleLayer,
#     'PrimaryCap': PrimaryCap,
#     'squash': squash,
#     'capsule_length': capsule_length,
#     'margin_loss': margin_loss,  # include if you're using a custom margin loss

# }
model=tf.keras.models.load_model(r"D:\Major Project\Final Proper\drowsiness_lesser_india_plus_eye.keras",custom_objects={"CapsuleLayer": CapsuleLayer,"Length":Length,"margin_loss":margin_loss})

In [4]:
print(model.output_shape)


(None, 4)


In [ ]:
import os
import cv2

def extract_frames(video_path, output_folder, frame_interval=10):
    os.makedirs(output_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    
    frame_count = 0
    saved_count = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        print(f"Frame {saved_count}: Shape={frame.shape}")  # Debugging print

        if True:  # Force rotation for every frame
            print(f"🔄 Rotating frame {saved_count}...")
            frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)

        frame_path = os.path.join(output_folder, f"frame_{saved_count}.jpg")
        cv2.imwrite(frame_path, frame)
        saved_count += 1

        frame_count += 1
    
    cap.release()
    cv2.destroyAllWindows()
    print(f"✅ Extracted {saved_count} frames with corrected orientation to {output_folder}")

# **Run it on your video**
video_path = r"D:\Major Project\Final Proper\Video_Data\Fold1_part2\07\10.mp4"  # Update this
output_folder = "american_extracted_frames"
extract_frames(video_path, output_folder)


In [ ]:
import os
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
import shutil

# **Load Your Pretrained Model**
#model = tf.keras.models.load_model("your_model_path.h5")  # Change to your model path

# **Define Class Labels**
CLASS_LABELS = {
    0: "Eyes Closed",  # Drowsy
    1: "Happy",        # Non-Drowsy
    2: "Neutral",      # Non-Drowsy
    3: "Yawning"       # Drowsy
}

# **Setup MediaPipe Face Mesh**
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)

# **Dataset Paths**
input_folder = "american_extracted_frames"
drowsy_folder = "drowsy"
non_drowsy_folder = "non_drowsy"
uncertain_folder = "uncertain"

os.makedirs(drowsy_folder, exist_ok=True)
os.makedirs(non_drowsy_folder, exist_ok=True)
os.makedirs(uncertain_folder, exist_ok=True)

# **Preprocessing Function for Model**
def preprocess_image(image_path, target_size=(224, 224)):
    img = cv2.imread(image_path)
    #img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, target_size)
    img = img / 255.0  # Normalize
    return np.expand_dims(img, axis=0)  # Add batch dimension

# **Function to Check Head Orientation**
def get_head_tilt(face_landmarks, image_width):
    left_cheek = face_landmarks.landmark[234].x * image_width
    right_cheek = face_landmarks.landmark[454].x * image_width
    face_center = (left_cheek + right_cheek) / 2

    if face_center < image_width * 0.35:
        return "Right"
    elif face_center > image_width * 0.65:
        return "Left"
    else:
        return "Center"

# **Function to Process Image and Classify Drowsiness**
def classify_drowsiness(img, img_path):
    h, w, _ = img.shape
    rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_img)

    avg_eye_opening = None
    mouth_opening = None
    head_orientation = "Center"

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            head_orientation = get_head_tilt(face_landmarks, w)

            # **Extract Eye Landmarks**
            left_eye_top = face_landmarks.landmark[159]
            left_eye_bottom = face_landmarks.landmark[145]
            right_eye_top = face_landmarks.landmark[386]
            right_eye_bottom = face_landmarks.landmark[374]

            if head_orientation == "Right":
                avg_eye_opening = (left_eye_bottom.y - left_eye_top.y) * h
            elif head_orientation == "Left":
                avg_eye_opening = (right_eye_bottom.y - right_eye_top.y) * h
            else:
                avg_eye_opening = ((left_eye_bottom.y - left_eye_top.y) + (right_eye_bottom.y - right_eye_top.y)) * h / 2

            # **Mouth Landmarks**
            upper_lip_top = face_landmarks.landmark[13]
            lower_lip_bottom = face_landmarks.landmark[14]
            mouth_opening = (lower_lip_bottom.y - upper_lip_top.y) * h

    # **Preprocess Image for Model**
    img_array = preprocess_image(img_path)

    # **Model Prediction**
    prediction = model.predict(img_array)
    class_id = np.argmax(prediction)
    confidence = round(max(prediction[0]) * 100, 2)

    # **Fusion of MediaPipe & Model Results**
    eye_threshold = 5
    mouth_threshold = 15

    if avg_eye_opening is not None and avg_eye_opening < eye_threshold:
        detected_drowsy = True
    elif mouth_opening is not None and mouth_opening > mouth_threshold:
        detected_drowsy = True
    else:
        detected_drowsy = class_id in [0, 3]

    # **Final Decision**
    if detected_drowsy:
        target_folder = drowsy_folder
        status = "Drowsy"
        color = (0, 0, 255)
    else:
        target_folder = non_drowsy_folder
        status = "Non-Drowsy"
        color = (0, 255, 0)

    # **Store Image in Folder Based on Confidence**
    if confidence >= 75:
        print(f"✅ Image: {img_path} → {CLASS_LABELS[class_id]} ({status}) (Confidence: {confidence}%)")
        shutil.copy2(img_path, os.path.join(target_folder, os.path.basename(img_path)))
    else:
        print(f"⚠️ Uncertain: {img_path} (Confidence: {confidence}%) - Below threshold")
        shutil.copy2(img_path, os.path.join(uncertain_folder, os.path.basename(img_path)))

# **Process All Images in Folder**
for filename in os.listdir(input_folder):
    img_path = os.path.join(input_folder, filename)
    if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
        continue

    img = cv2.imread(img_path)
    if img is not None:
        classify_drowsiness(img, img_path)
    else:
        print(f"⚠️ Skipping {filename} (Could not read image)")

print("\n✅ Classification completed! High-confidence images sorted, uncertain ones stored separately.")


In [7]:
# class_indices = model.class_indices
# print(f"Class Indices from Model: {class_indices}")


In [8]:
# import cv2
# import numpy as np
# import tensorflow as tf


# # Image preprocessing function
# def preprocess_image(image_path, target_size=(224, 224)):
#     img = cv2.imread(image_path)  # Read the image
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
#     img = cv2.resize(img, target_size)  # Resize to model input size
#     img = img / 255.0  # Normalize pixel values (0 to 1)
#     return np.expand_dims(img, axis=0)  # Add batch dimension

# # Function to classify a single image
# def classify_image(image_path, confidence_threshold=75):
#     img_array = preprocess_image(image_path)

#     # Predict using the model
#     prediction = model.predict(img_array)
#     class_id = np.argmax(prediction)  # Get the predicted class index
#     confidence = round(max(prediction[0]) * 100, 2)  # Get highest probability

#     # **Corrected Mapping: Closed Eyes (0) = Drowsy, Open Eyes (1) = Non-Drowsy**
#     label = "Drowsy 😴" if class_id == 0 else "Non-Drowsy 😃"

#     # Print result
#     print(f"🖼️ Image: {image_path}")
#     print(f"🔍 Prediction: {label} (Confidence: {confidence}%)")

#     # Return results
#     return {"image": image_path, "label": label, "confidence": confidence}

# # Test the function with an image
# image_path = r"D:\Major Project\Final Proper\Dataset\American\Drowsy\img_e_15341.jpg"# Change this to your image file path
# result = classify_image(image_path)



In [9]:
# import cv2
# import numpy as np
# import tensorflow as tf

# # **Define Class Labels (Matching Your Dataset)**
# CLASS_LABELS = {
#     0: "Eyes Closed",   # Drowsy (Mapped from "Eyeclose")
#     1: "Happy",         # Non-Drowsy
#     2: "Neutral",       # Non-Drowsy
#     3: "Yawning"        # Drowsy (Mapped from "Yawn")
# }

# # **Preprocess Image Before Prediction**
# def preprocess_image(image_path, target_size=(224, 224)):
#     img = cv2.imread(image_path)  # Read the image
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
#     img = cv2.resize(img, target_size)  # Resize to match model input
#     img = img / 255.0  # Normalize pixel values (0 to 1)
#     return np.expand_dims(img, axis=0)  # Add batch dimension

# # **Classify Image Function**
# def classify_image(image_path):
#     img_array = preprocess_image(image_path)

#     # Predict using the model
#     prediction = model.predict(img_array)
#     class_id = np.argmax(prediction)  # Get the predicted class index
#     confidence = round(max(prediction[0])*10 , 2)  # Get highest probability

#     # Determine drowsiness status
#     if class_id in [0, 3]:  # Eyes Closed or Yawning
#         status = "Drowsy 😴"
#     else:  # Happy or Neutral
#         status = "Non-Drowsy 😃"

#     # Print result
#     print(f"🖼️ Image: {image_path}")
#     print(f"🔍 Prediction: {CLASS_LABELS[class_id]} ({status}) (Confidence: {confidence}%)")

#     # Return results
#     return {"image": image_path, "label": CLASS_LABELS[class_id], "status": status, "confidence": confidence}

# # **Example Usage**
# image_path = r"D:\Major Project\Dataset_Classified\non_drowsy\Drowsy (4458).jpg" # Change to your test image path
# result = classify_image(image_path)


In [10]:
#NOT USING MEDIAPIPE
# import os
# import cv2
# import numpy as np
# import tensorflow as tf
# import shutil


# # **Define Dataset Paths**
# input_folder = "D:/Major Project/Dataset"  # Root dataset folder
# drowsy_folder = "D:/Major Project/Dataset_Classified/drowsy"
# non_drowsy_folder = "D:/Major Project/Dataset_Classified/non_drowsy"
# uncertain_folder = "D:/Major Project/Dataset_Classified/uncertain"  # Low-confidence images

# # **Create Output Folders if They Don't Exist**
# os.makedirs(drowsy_folder, exist_ok=True)
# os.makedirs(non_drowsy_folder, exist_ok=True)
# os.makedirs(uncertain_folder, exist_ok=True)

# # **Class Labels Mapping (Based on Your Dataset)**
# CLASS_LABELS = {
#     0: "Eyes Closed",   # Drowsy
#     1: "Happy",         # Non-Drowsy
#     2: "Neutral",       # Non-Drowsy
#     3: "Yawning"        # Drowsy
# }

# # **Preprocessing Function**
# def preprocess_image(image_path, target_size=(224, 224)):
#     img = cv2.imread(image_path)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     img = cv2.resize(img, target_size)
#     img = img / 255.0  # Normalize
#     return np.expand_dims(img, axis=0)  # Add batch dimension

# # **Classify and Sort Images**
# CONFIDENCE_THRESHOLD = 75  # Confidence cutoff for certainty

# for subdir, _, files in os.walk(input_folder):
#     for filename in files:
#         img_path = os.path.join(subdir, filename)

#         # **Skip Non-Image Files**
#         if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
#             continue

#         img_array = preprocess_image(img_path)

#         # **Predict Using Model**
#         prediction = model.predict(img_array)
#         class_id = np.argmax(prediction)  # Get class index
#         confidence = round(max(prediction[0]) * 100, 2)  # Get highest probability

#         # **Determine Drowsiness Status**
#         if class_id in [0, 3]:  # Eyes Closed or Yawning
#             status = "Drowsy"
#             target_folder = drowsy_folder
#         else:  # Happy or Neutral
#             status = "Non-Drowsy"
#             target_folder = non_drowsy_folder

#         # **Sort Based on Confidence Level**
#         if confidence >= CONFIDENCE_THRESHOLD:
#             print(f"✅ {filename} → {CLASS_LABELS[class_id]} ({status}) (Confidence: {confidence}%)")
#             shutil.copy2(img_path, os.path.join(target_folder, filename))
#         else:
#             print(f"⚠️ Uncertain: {filename} (Confidence: {confidence}%) - Below threshold")
#             shutil.copy2(img_path, os.path.join(uncertain_folder, filename))

# print("\n✅ Classification completed! High-confidence images sorted, uncertain ones stored separately.")


##USING MEDIAPIPE

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import shutil
import mediapipe as mp

# **Load MediaPipe Face Mesh**
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5)

# **Load Pretrained Model**
model = model# Ensure the path is correct
print("✅ Model loaded successfully!")

# **Get Input Folder from User**
input_folder = r"D:\Major Project\Dataset\notdrowsy"  # Update the input directory

# **Check if the directory exists**
if not os.path.exists(input_folder):
    print(f"❌ Error: The directory '{input_folder}' does not exist.")
    exit()

# **Define Output Folders**
output_base = os.path.join(input_folder, "Dataset_Classified")
drowsy_folder = os.path.join(output_base, "drowsy")
non_drowsy_folder = os.path.join(output_base, "non_drowsy")
uncertain_folder = os.path.join(output_base, "uncertain")

# **Create Output Folders if They Don't Exist**
os.makedirs(drowsy_folder, exist_ok=True)
os.makedirs(non_drowsy_folder, exist_ok=True)
os.makedirs(uncertain_folder, exist_ok=True)

# **Class Labels Mapping**
CLASS_LABELS = {
    0: "Eyes Closed",   # Drowsy
    1: "Happy",         # Non-Drowsy
    2: "Neutral",       # Non-Drowsy
    3: "Yawning"        # Drowsy
}

# **Preprocessing Function**
def preprocess_image(img, target_size=(224, 224)):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, target_size)
    img = img / 255.0  # Normalize
    return np.expand_dims(img, axis=0)  # Add batch dimension

# **Confidence Threshold**
CONFIDENCE_THRESHOLD = 75  

# **Process Images in the Input Folder**
for subdir, _, files in os.walk(input_folder):
    for filename in files:
        img_path = os.path.join(subdir, filename)

        # **Skip Non-Image Files**
        if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            continue

        img = cv2.imread(img_path)
        if img is None:
            print(f"⚠️ Skipping {filename} (Could not read image)")
            continue

        h, w, _ = img.shape

        # **Detect face mesh using MediaPipe**
        rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(rgb_img)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # **Extract Eye Landmarks**
                left_eye_top = face_landmarks.landmark[159]
                left_eye_bottom = face_landmarks.landmark[145]
                right_eye_top = face_landmarks.landmark[386]
                right_eye_bottom = face_landmarks.landmark[374]

                left_eye_opening = int((left_eye_bottom.y - left_eye_top.y) * h)
                right_eye_opening = int((right_eye_bottom.y - right_eye_top.y) * h)

                # **Eye openness threshold**
                eye_threshold = 5  # Adjust based on testing
                avg_eye_opening = (left_eye_opening + right_eye_opening) / 2

                # **Extract Mouth Landmarks**
                upper_lip_top = face_landmarks.landmark[13]
                lower_lip_bottom = face_landmarks.landmark[14]

                mouth_opening = int((lower_lip_bottom.y - upper_lip_top.y) * h)

                # **Preprocess Image & Predict Using Model**
                img_array = preprocess_image(img)
                prediction = model.predict(img_array)  # Model prediction
                class_id = np.argmax(prediction)  # Get class index
                confidence = round(max(prediction[0]) * 100, 2)  # Get highest probability

                # **Drowsiness Classification Logic**
                if avg_eye_opening < eye_threshold or class_id == 0:
                    status = "Drowsy"
                    target_folder = drowsy_folder
                elif mouth_opening > 15 and class_id == 3:
                    status = "Drowsy"
                    target_folder = drowsy_folder
                else:
                    status = "Non-Drowsy"
                    target_folder = non_drowsy_folder

                # **Sort Based on Confidence Level**
                if confidence >= CONFIDENCE_THRESHOLD:
                    print(f"✅ {filename} → {CLASS_LABELS[class_id]} ({status}) (Confidence: {confidence}%)")
                    shutil.copy2(img_path, os.path.join(target_folder, filename))
                else:
                    print(f"⚠️ Uncertain: {filename} (Confidence: {confidence}%) - Below threshold")
                    shutil.copy2(img_path, os.path.join(uncertain_folder, filename))
        else:
            print(f"🚫 No face detected in {filename}. Marking as uncertain.")
            shutil.copy2(img_path, os.path.join(uncertain_folder, filename))

print("\n✅ Classification completed! High-confidence images sorted, uncertain ones stored separately.")


In [ ]:
# import os
# import cv2
# import numpy as np
# import tensorflow as tf
# import shutil
# import mediapipe as mp

# # **Load MediaPipe Face Mesh**
# mp_face_mesh = mp.solutions.face_mesh
# face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5)

# # **Load Pretrained Model**
# #model = tf.keras.models.load_model("D:/Major Project/your_model.h5")  # Update with actual model path

# # **Define Dataset Paths**
# input_folder = "D:/Major Project/Dataset"  # Root dataset folder
# drowsy_folder = "D:/Major Project/Dataset_Classified/drowsy"
# non_drowsy_folder = "D:/Major Project/Dataset_Classified/non_drowsy"
# uncertain_folder = "D:/Major Project/Dataset_Classified/uncertain"  # Low-confidence images

# # **Create Output Folders if They Don't Exist**
# os.makedirs(drowsy_folder, exist_ok=True)
# os.makedirs(non_drowsy_folder, exist_ok=True)
# os.makedirs(uncertain_folder, exist_ok=True)

# # **Class Labels Mapping (Based on Your Dataset)**
# CLASS_LABELS = {
#     0: "Eyes Closed",   # Drowsy
#     1: "Happy",         # Non-Drowsy
#     2: "Neutral",       # Non-Drowsy
#     3: "Yawning"        # Drowsy
# }

# # **Preprocessing Function**
# def preprocess_image(img, target_size=(224, 224)):
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     img = cv2.resize(img, target_size)
#     img = img / 255.0  # Normalize
#     return np.expand_dims(img, axis=0)  # Add batch dimension

# # **Classify and Sort Images**
# CONFIDENCE_THRESHOLD = 75  # Confidence cutoff for certainty

# for subdir, _, files in os.walk(input_folder):
#     for filename in files:
#         img_path = os.path.join(subdir, filename)

#         # **Skip Non-Image Files**
#         if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
#             continue

#         img = cv2.imread(img_path)
#         if img is None:
#             print(f"⚠️ Skipping {filename} (Could not read image)")
#             continue

#         h, w, _ = img.shape

#         # **Detect face mesh using MediaPipe**
#         rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#         results = face_mesh.process(rgb_img)

#         if results.multi_face_landmarks:
#             for face_landmarks in results.multi_face_landmarks:
#                 # **Extract Eye Landmarks**
#                 left_eye_top = face_landmarks.landmark[159]
#                 left_eye_bottom = face_landmarks.landmark[145]
#                 right_eye_top = face_landmarks.landmark[386]
#                 right_eye_bottom = face_landmarks.landmark[374]

#                 left_eye_opening = int((left_eye_bottom.y - left_eye_top.y) * h)
#                 right_eye_opening = int((right_eye_bottom.y - right_eye_top.y) * h)

#                 # **Eye openness threshold**
#                 eye_threshold = 5  # Adjust based on testing
#                 avg_eye_opening = (left_eye_opening + right_eye_opening) / 2

#                 # **Extract Mouth Landmarks**
#                 upper_lip_top = face_landmarks.landmark[13]
#                 lower_lip_bottom = face_landmarks.landmark[14]

#                 mouth_opening = int((lower_lip_bottom.y - upper_lip_top.y) * h)

#                 # **Preprocess and Predict Using Model**
#                 img_array = preprocess_image(img)
#                 prediction = model.predict(img_array)
#                 class_id = np.argmax(prediction)  # Get class index
#                 confidence = round(max(prediction[0]) * 100, 2)  # Get highest probability

#                 # **Drowsiness Classification Logic**
#                 if avg_eye_opening < eye_threshold or class_id == 0:
#                     status = "Drowsy"
#                     target_folder = drowsy_folder
#                 elif mouth_opening > 15 and class_id == 3:
#                     status = "Drowsy"
#                     target_folder = drowsy_folder
#                 else:
#                     status = "Non-Drowsy"
#                     target_folder = non_drowsy_folder

#                 # **Sort Based on Confidence Level**
#                 if confidence >= CONFIDENCE_THRESHOLD:
#                     print(f"✅ {filename} → {CLASS_LABELS[class_id]} ({status}) (Confidence: {confidence}%)")
#                     shutil.copy2(img_path, os.path.join(target_folder, filename))
#                 else:
#                     print(f"⚠️ Uncertain: {filename} (Confidence: {confidence}%) - Below threshold")
#                     shutil.copy2(img_path, os.path.join(uncertain_folder, filename))
#         else:
#             print(f"🚫 No face detected in {filename}. Marking as uncertain.")
#             shutil.copy2(img_path, os.path.join(uncertain_folder, filename))

# print("\n✅ Classification completed! High-confidence images sorted, uncertain ones stored separately.")


In [ ]:
!pip install mediapipe 

In [ ]:
# ########---------------------------WORKIG ONE------------------------------------#######
# import cv2
# import numpy as np
# import mediapipe as mp
# import tensorflow as tf

# # **Load MediaPipe Face Mesh**
# mp_face_mesh = mp.solutions.face_mesh
# face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# # **Define class labels**
# CLASS_LABELS = {
#     0: "Eyes Closed",  # Drowsy
#     1: "Happy",        # Non-Drowsy
#     2: "Neutral",      # Non-Drowsy
#     3: "Yawning"       # Drowsy
# }

# # **Start the webcam**
# cap = cv2.VideoCapture(0)  # 0 for default camera

# # **Function to preprocess the frame for model input**
# def preprocess_frame(frame, target_size=(224, 224)):
#     img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     img = cv2.resize(img, target_size)
#     img = img / 255.0  # Normalize pixel values (0 to 1)
#     return np.expand_dims(img, axis=0)  # Add batch dimension

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         print("❌ Error: Unable to capture video")
#         break

#     # **Convert frame to RGB for MediaPipe**
#     rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     results = face_mesh.process(rgb_frame)

#     if results.multi_face_landmarks:
#         for face_landmarks in results.multi_face_landmarks:
#             h, w, _ = frame.shape

#             # **Get eye landmarks (based on MediaPipe Face Mesh indices)**
#             left_eye_top = face_landmarks.landmark[159]  # Upper eyelid
#             left_eye_bottom = face_landmarks.landmark[145]  # Lower eyelid

#             right_eye_top = face_landmarks.landmark[386]  # Upper eyelid
#             right_eye_bottom = face_landmarks.landmark[374]  # Lower eyelid

#             # Convert normalized coordinates to pixels
#             left_eye_opening = int((left_eye_bottom.y - left_eye_top.y) * h)
#             right_eye_opening = int((right_eye_bottom.y - right_eye_top.y) * h)

#             # **Calculate average eye openness**
#             avg_eye_opening = (left_eye_opening + right_eye_opening) / 2

#             # **Get mouth landmarks**
#             upper_lip_top = face_landmarks.landmark[13]  
#             lower_lip_bottom = face_landmarks.landmark[14]  

#             upper_y = int(upper_lip_top.y * h)
#             lower_y = int(lower_lip_bottom.y * h)

#             # **Calculate mouth opening**
#             mouth_opening = lower_y - upper_y

#             # **Preprocess and predict using the model**
#             img_array = preprocess_frame(frame)
#             prediction = model.predict(img_array)
#             class_id = np.argmax(prediction)  # Get class index
#             confidence = round(max(prediction[0]) * 100, 2)  # Get highest probability

#             # **Drowsiness Classification Logic**
#             if avg_eye_opening < 5 or class_id == 0:  # If eyes are too closed OR model says eyes closed
#                 status = "Drowsy 😴 (Eyes Closed)"
#                 color = (0, 0, 255)  # Red for drowsy
#             elif mouth_opening > 15 and class_id == 3:  # If mouth is open + model says yawning
#                 status = "Drowsy 😴 (Yawning)"
#                 color = (0, 0, 255)
#             else:
#                 status = "Non-Drowsy 😊"
#                 color = (0, 255, 0)  # Green for non-drowsy

#             # **Display Results on Screen**
#             text = f"{CLASS_LABELS[class_id]} ({status}) - {confidence}%"
#             cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

#     # **Show Webcam Output**
#     cv2.imshow("Drowsiness Detection", frame)

#     # **Press 'q' to exit**
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # **Release resources**
# cap.release()
# cv2.destroyAllWindows()


# ########---------WORKING Perfectly----------#######

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf

# **Load MediaPipe Face Mesh**
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# **Define class labels**
CLASS_LABELS = {
    0: "Eyes Closed",  # Drowsy
    1: "Happy",        # Non-Drowsy
    2: "Neutral",      # Non-Drowsy
    3: "Yawning"       # Drowsy
}

# **Start the webcam**
cap = cv2.VideoCapture(0)  # 0 for default camera

# **Function to preprocess the frame for model input**
def preprocess_frame(frame, target_size=(224, 224)):
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, target_size)
    img = img / 255.0  # Normalize pixel values (0 to 1)
    return np.expand_dims(img, axis=0)  # Add batch dimension

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Error: Unable to capture video")
        break

    # **Convert frame to RGB for MediaPipe**
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            h, w, _ = frame.shape

            # **Get eye landmarks**
            left_eye_top = face_landmarks.landmark[159]  
            left_eye_bottom = face_landmarks.landmark[145]  
            right_eye_top = face_landmarks.landmark[386]  
            right_eye_bottom = face_landmarks.landmark[374]  

            # Convert normalized coordinates to pixels
            left_eye_opening = int((left_eye_bottom.y - left_eye_top.y) * h)
            right_eye_opening = int((right_eye_bottom.y - right_eye_top.y) * h)

            # **Set optimized eye openness threshold**
            eye_threshold = 5  # Adjust this based on testing

            # **Calculate average eye openness**
            avg_eye_opening = (left_eye_opening + right_eye_opening) / 2

            # **Get mouth landmarks**
            upper_lip_top = face_landmarks.landmark[13]  
            lower_lip_bottom = face_landmarks.landmark[14]  

            # Convert to pixels
            upper_y = int(upper_lip_top.y * h)
            lower_y = int(lower_lip_bottom.y * h)

            # **Calculate mouth opening**
            mouth_opening = lower_y - upper_y

            # **Preprocess and predict using the model**
            img_array = preprocess_frame(frame)
            prediction = model.predict(img_array)
            class_id = np.argmax(prediction)  # Get class index
            confidence = round(max(prediction[0]) * 100, 2)  # Get highest probability

            # **Drowsiness Classification Logic**
            if avg_eye_opening < eye_threshold or class_id == 0:
                status = "Drowsy 😴 (Eyes Closed)"
                color = (0, 0, 255)  # Red for drowsy
            elif mouth_opening > 15 and class_id == 3:
                status = "Drowsy 😴 (Yawning)"
                color = (0, 0, 255)
            else:
                status = "Non-Drowsy 😊"
                color = (0, 255, 0)  # Green for non-drowsy

            # **Display Results on Screen**
            text = f"{CLASS_LABELS[class_id]} ({status}) - {confidence}%"
            cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    # **Show Webcam Output**
    cv2.imshow("Drowsiness Detection", frame)

    # **Press 'q' to exit**
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# **Release resources**
cap.release()
cv2.destroyAllWindows()

In [ ]:
# import cv2
# import numpy as np
# import tensorflow as tf

# # **Define class labels**
# CLASS_LABELS = {
#     0: "Eyes Closed",  # Drowsy
#     1: "Happy",        # Non-Drowsy
#     2: "Neutral",      # Non-Drowsy
#     3: "Yawning"       # Drowsy
# }

# # **Load Haarcascade Classifiers**
# face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
# eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")
# mouth_cascade = cv2.CascadeClassifier(r"D:\Major Project\Final Proper\haarcascade_mcs_mouth.xml")  # Mouth detection

# # **Check if mouth cascade is loaded**
# if mouth_cascade.empty():
#     print("❌ Error: Mouth cascade file not loaded! Check the file path.")
#     exit()
# else:
#     print("✅ Mouth cascade loaded successfully!")

# # **Start the webcam**
# cap = cv2.VideoCapture(0)  # 0 for default camera

# # **Function to preprocess the frame for model input**
# def preprocess_frame(frame, target_size=(224, 224)):
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     gray = cv2.equalizeHist(gray)  # Improve contrast
#     img = cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB)  # Convert back to RGB
#     img = cv2.resize(img, target_size)
#     img = img / 255.0  # Normalize pixel values (0 to 1)
#     return np.expand_dims(img, axis=0)  # Add batch dimension

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         print("❌ Error: Unable to capture video")
#         break

#     # **Face detection**
#     faces = face_cascade.detectMultiScale(frame, scaleFactor=1.3, minNeighbors=5)
    
#     for (x, y, w, h) in faces:
#         roi_face = frame[y:y+h, x:x+w]  # Extract face region
        
#         # **Eye detection**
#         eyes = eye_cascade.detectMultiScale(roi_face)
        
#         # **Mouth detection (focus on lower 40% of the face)**
#         mouth_y_start = int(0.6 * h)  # Start from 60% down
#         mouth_region = roi_face[int(h * 0.4):h, :]
        
#         mouths = mouth_cascade.detectMultiScale(mouth_region, scaleFactor=1.1, minNeighbors=5)

#         # **Preprocess and predict using the model**
#         img_array = preprocess_frame(roi_face)
#         prediction = model.predict(img_array)
#         class_id = np.argmax(prediction)  # Get class index
#         confidence = round(max(prediction[0]) * 100, 2)  # Get highest probability

#         # **Debugging: Print raw predictions**
#         print(f"🔍 Raw Prediction: {prediction}")

#         # **Drowsiness Classification Logic**
#         if class_id in [0, 3]:  # Eyes Closed OR Yawning
#             status = "Drowsy 😴"
#             color = (0, 0, 255)  # Red for drowsy
#         else:
#             status = "Non-Drowsy 😊"
#             color = (0, 255, 0)  # Green for non-drowsy

#         # **Additional Check: If no eyes detected, assume Drowsy**
#         if len(eyes) == 0:
#             status = "Drowsy 😴 (Eyes Not Detected)"
#             color = (0, 0, 255)

#         # **Yawn Detection: If mouth is detected and model predicts 'Yawning'**
#         if len(mouths) > 0 and class_id == 3:
#             status = "Drowsy 😴 (Yawning Detected)"
#             color = (0, 0, 255)

#         # **Display Results on Screen**
#         text = f"{CLASS_LABELS[class_id]} ({status}) - {confidence}%"
#         cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
#         cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

#     # **Show Webcam Output**
#     cv2.imshow("Drowsiness Detection", frame)

#     # **Press 'q' to exit**
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # **Release resources**
# cap.release()
# cv2.destroyAllWindows()
